# Census Income - Proyect with Random Forest (Modelo 3)

> Abordamos el tema del desbalanceo de datos probando diferentes iteraciones del hiperparametro 'class_weight'

Variables del conjunto de datos "Census Income" de UCI. Este conjunto de datos se utiliza comúnmente para tareas de clasificación, como predecir si una persona gana más o menos de $50,000 al año, en base a sus características. Las caracteristicas son:

1. **age** (edad): La edad de la persona en años.
2. **workclass** (tipo de trabajo): Clasificación del tipo de empleo (e.g., asalariado, autónomo, gobierno).
3. **fnlwgt** (ponderación final): Peso final calculado para las muestras en la encuesta, representa el número de personas que una fila puede representar.
4. **education** (educación): Nivel educativo alcanzado (e.g., escuela secundaria, licenciatura).
5. **education-num** (número de años de educación): Número de años de educación recibidos.
6. **marital-status** (estado civil): Estado civil de la persona (e.g., casado, soltero).
7. **occupation** (ocupación): Tipo de ocupación o profesión de la persona.
8. **relationship** (relación): Relación de la persona con el jefe de familia (e.g., esposo, esposa, hijo).
9. **race** (raza): Clasificación racial de la persona (e.g., blanca, negra, asiática).
10. **sex** (sexo): Sexo de la persona (masculino o femenino).
11. **capital-gain** (ganancia de capital): Ganancias de capital obtenidas por inversiones.
12. **capital-loss** (pérdida de capital): Pérdidas de capital incurridas.
13. **hours-per-week** (horas por semana): Número de horas trabajadas por semana.
14. **native-country** (país de origen): País de nacimiento de la persona.
15. **income** (ingreso): Ingreso anual, clasificado en ">50K" o "<=50K", indicando si la persona gana más o menos de $50,000 al año.



In [21]:
import pandas as pd

# Cargar el archivo de datos principal 'adult.data'
data_path = './adult.data'
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 
                'marital_status', 'occupation', 'relationship', 'race', 
                'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 
                'native_country', 'income']

# Cargar el dataset en un DataFrame
df = pd.read_csv(data_path, header=None, names=column_names, engine='python')

# Mostrar las primeras filas
df.head(2)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [22]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder

X = df.drop(['income','native_country','race'], axis=1)
y = df['income']

ord_enc = OrdinalEncoder()
X = pd.DataFrame(ord_enc.fit_transform(X), columns=X.columns)


In [23]:

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)

rf = RandomForestClassifier(bootstrap=True,
                            max_depth=20,
                            max_features= 5,
                            min_samples_leaf = 6,
                            min_samples_split= 2,
                            n_estimators= 200,
                            # class_weight = {' <=50K': 0.7, ' >50K': 1},

    random_state=42)
rf
# entrenamos el modelo
rf.fit(x_train, y_train)
rf

RandomForestClassifier(max_depth=20, max_features=5, min_samples_leaf=6,
                       n_estimators=200, random_state=42)

In [24]:
from sklearn.metrics import classification_report
y_pred = rf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       <=50K       0.89      0.94      0.91      7447
        >50K       0.77      0.62      0.68      2322

    accuracy                           0.86      9769
   macro avg       0.83      0.78      0.80      9769
weighted avg       0.86      0.86      0.86      9769



In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[7009,  438],
       [ 891, 1431]], dtype=int64)

## Abordando el tema del desbalanceo de la variable objetivo

Buscamos penalizar a la clase mayoritaria dandole menos peso o importancia al momento del entrenamiento. Para eso nos apoyamos en el hiperparametro 'class_weight

In [26]:
rf2 = RandomForestClassifier(bootstrap=True,
                            max_depth=20,
                            max_features= 5,
                            min_samples_leaf = 6,
                            min_samples_split= 2,
                            n_estimators= 200,
                            # class_weight = {' <=50K': 0.7, ' >50K': 1},

    random_state=42)


param_grid = {
    'class_weight':[{' <=50K': 0.1*i, ' >50K':1.1} for i in range(10)]

}

grid_search = GridSearchCV(estimator=rf2, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

grid_search.fit(x_train, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(max_depth=20, max_features=5,
                                              min_samples_leaf=6,
                                              n_estimators=200,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'class_weight': [{' <=50K': 0.0, ' >50K': 1.1},
                                          {' <=50K': 0.1, ' >50K': 1.1},
                                          {' <=50K': 0.2, ' >50K': 1.1},
                                          {' <=50K': 0.30000000000000004,
                                           ' >50K': 1.1},
                                          {' <=50K': 0.4, ' >50K': 1.1},
                                          {' <=50K': 0.5, ' >50K': 1.1},
                                          {' <=50K': 0.6000000000000001,
                                           ' >50K': 1.1},
                                          {' <=50K': 0.7000000000000001,
                                           ' >50K': 1.1},
                                          {' <=50K': 0.8, ' >50K': 1.1},
                                          {' <=50K': 0.9, ' >50K': 1.1}]},
             scoring='accuracy', verbose=2)

In [27]:

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

best_rf2 = grid_search.best_estimator_

# Predicciones en el conjunto de prueba
y_pred = best_rf2.predict(x_test)

# Calcular precisión
accuracy = accuracy_score(y_test, y_pred)
print("Precisión en el conjunto de prueba:", accuracy)

Mejores parámetros: {'class_weight': {' <=50K': 0.9, ' >50K': 1.1}}
Mejor puntuación de validación cruzada: 0.8607844294254083
Precisión en el conjunto de prueba: 0.8605793837649708


In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       <=50K       0.90      0.93      0.91      7447
        >50K       0.73      0.65      0.69      2322

    accuracy                           0.86      9769
   macro avg       0.81      0.79      0.80      9769
weighted avg       0.86      0.86      0.86      9769



In [29]:
confusion_matrix(y_test, y_pred)

array([[6893,  554],
       [ 808, 1514]], dtype=int64)

In [30]:
y_pred_originals = rf.predict(X)
print(classification_report(y, y_pred_originals))

              precision    recall  f1-score   support

       <=50K       0.90      0.96      0.93     24720
        >50K       0.84      0.68      0.75      7841

    accuracy                           0.89     32561
   macro avg       0.87      0.82      0.84     32561
weighted avg       0.89      0.89      0.89     32561



In [31]:
confusion_matrix(y, y_pred_originals)

array([[23662,  1058],
       [ 2484,  5357]], dtype=int64)

## Conclusiones

- Diferentes interacciones en el manejo de hiperparametros para la clase minoritaria solo nos brindaron un aumento en el recall de esta, pero se perdio precision ademas de que el modelo bajo su performance reconociendo la clase mayoritaria por que se podria concluir que no se compensa lo que se gano con lo que se perdio 
- Se concluyo por no optar por un manejo de la clase desbalanceada usando la modificacion de 'class_weihgt', por el contrario se tratara de balancear la clase minoritaria directamente con los datos de entrenamiento